## CVX 2021-2022 Project 5 

In [70]:
# Import packages.
import cvxpy as cp
import numpy as np
from numpy.linalg import matrix_power
from numpy import linalg as LA

### Definitions

In [6]:
def scd_moment(Q0, Qw, A, k):
    temp = np.zeros(Q0.shape)
    for i in range(k + 1):
        temp += A**(k-i) @ Qw @ (A**(k-i)).T
        
    return A**k @ Q0 @ (A**k).T + temp

In [18]:
# Define the dynamics 
A = np.matrix([[0.5, 0.8], [0, 0.5]])
n = 2
Q0 = np.matrix([[1, 0], [0, 1]])
Qw = np.matrix([[1, 0], [0, 1]])

# Compute the first two moments 
K = 10
M = scd_moment(Q0, Qw, A, K + 1)

### Empirical Probability

In [15]:
# The given data contains for E trajectories (different initial conditions) the vector x_{K+1}.
# load that data 
data = np.loadtxt("datax.txt")

# compute the empirical probability of being in the safe set 
E = 1000
rad = 8

t = np.linalg.norm(data, axis = 0, ord=1)
emp_proba = (t < rad).sum() / E
print(emp_proba)

0.998


### SDP Formulation

In [66]:
b = 1/2 * np.array([[1.0, 1],
                    [1, -1],
                    [-1, 1],
                    [-1, -1]])

c = np.array([-rad * 1.0,-rad, -rad, -rad])

P = cp.Variable((n,n), symmetric=True)
q = cp.Variable((n, 1))
r = cp.Variable((1,1))
tau = cp.Variable(len(c))


obj = 1 - cp.trace(M @ P) - r

constraints = []
constraints.append(tau >= 0)

for i in range(len(c)):
    temp1 = cp.hstack([P, q - tau[i]*b[i].reshape(-1, 1)])
    temp2 = cp.vstack([q - tau[i]*b[i].reshape(-1, 1), (r - 1 - tau[i]*c[i].reshape(1,1))]).T
    ti = cp.vstack([temp1, temp2])
    constraints.append(ti >> 0)
    
    temp1 = cp.hstack([P, q])
    temp2 = cp.hstack([q.T, r])
    ti = cp.vstack([temp1, temp2])
    constraints.append(ti >> 0)
    
prob = cp.Problem(cp.Maximize(obj),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)

The optimal value is 0.863854713139716


### Discussion

Indeed, as expected, the sdp solution give us a lower value than the empirical probability value computed previously (the expectancy of the empirical value is around the ground truth). TODO